In [29]:
from azureml.core import Workspace, Datastore, Dataset
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.environment import Environment
from azureml.core import Experiment
from azureml.core.authentication import InteractiveLoginAuthentication


In [14]:
#ws=Workspace.create(name='hyperdive12',subscription_id='1db33695-8135-4616-9bb4-9574b401d454',resource_group='defaultsrg02',location='eastus2')
#auth=InteractiveLoginAuthentication(tenant_id='6ad91895-de06-485e-bc51-fce126cc8530'))

Deploying StorageAccount with name hyperdivstorage8b4c9561c.
Deploying AppInsights with name hyperdivinsights3fd0ae93.
Deployed AppInsights with name hyperdivinsights3fd0ae93. Took 2.4 seconds.
Deploying KeyVault with name hyperdivkeyvault359d3b2c.
Deployed KeyVault with name hyperdivkeyvault359d3b2c. Took 17.51 seconds.
Deployed StorageAccount with name hyperdivstorage8b4c9561c. Took 22.21 seconds.
Deploying Workspace with name hyperdive12.
Deployed Workspace with name hyperdive12. Took 22.19 seconds.


In [2]:
ws=Workspace.from_config('C:/Users/satya/Downloads/Azure/confighyper.json')

In [3]:
az_access=Datastore.register_azure_blob_container(workspace=ws,datastore_name='defaultsdatastore',container_name='defaults',account_name='hyperdrive122',
account_key='oVP/zSPRAu6HeTlxBRUc1fVqNsfj+DMk/xiw0EbXgMJCNsovu8hBosh4MxPFXrMxhg9D+pT0pSEn+AStJPPP2g==')

In [4]:
az_access=Datastore.get(ws,datastore_name='defaultsdatastore')

In [5]:
csv=[(az_access,'credit.csv')]
ds=Dataset.Tabular.from_delimited_files(path=csv)

In [10]:
dataset=ds.register(ws,ds,'credit')

In [26]:
dataset.to_pandas_dataframe().head().isnull().sum()

Column1                                 0
Unnamed: 0                              0
SeriousDlqin2yrs                        0
RevolvingUtilizationOfUnsecuredLines    0
age                                     0
NumberOfTime30-59DaysPastDueNotWorse    0
DebtRatio                               0
MonthlyIncome                           0
NumberOfOpenCreditLinesAndLoans         0
NumberOfTimes90DaysLate                 0
NumberRealEstateLoansOrLines            0
NumberOfTime60-89DaysPastDueNotWorse    0
NumberOfDependents                      0
dtype: int64

In [12]:
from azureml.core.compute import ComputeTarget, AmlCompute

In [18]:
#import conda depe
from azureml.core.conda_dependencies import CondaDependencies

envt=Environment(name='envt')
envtdep=CondaDependencies.create(conda_packages=['pandas','numpy','scikit-learn'])
envt.python.conda_dependencies=envtdep
envt.register(ws)

{
    "databricks": {
        "eggLibraries": [],
        "jarLibraries": [],
        "mavenLibraries": [],
        "pypiLibraries": [],
        "rcranLibraries": []
    },
    "docker": {
        "arguments": [],
        "baseDockerfile": null,
        "baseImage": "mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04:20220616.v1",
        "baseImageRegistry": {
            "address": null,
            "password": null,
            "registryIdentity": null,
            "username": null
        },
        "enabled": false,
        "platform": {
            "architecture": "amd64",
            "os": "Linux"
        },
        "sharedVolumes": true,
        "shmSize": null
    },
    "environmentVariables": {
        "EXAMPLE_ENV_VAR": "EXAMPLE_VALUE"
    },
    "inferencingStackVersion": null,
    "name": "envt",
    "python": {
        "baseCondaEnvironment": null,
        "condaDependencies": {
            "channels": [
                "anaconda",
                "conda-forge"
         

In [43]:
cluster_name='cluster1'
config=AmlCompute.provisioning_configuration(vm_size='STANDARD_D1',max_nodes=2)
cluster=AmlCompute.create(ws,cluster_name,config)

In [19]:
#import RunConfiguration
from azureml.core.runconfig import RunConfiguration
runconfig=RunConfiguration()
runconfig.target=cluster
runconfig.environment=envt

In [44]:
#import hyperdive from azure
from azureml.train.hyperdrive  import GridParameterSampling, choice

hyperparams=GridParameterSampling({'--n_neighbors':choice(5,10,15,20)})

In [49]:
from azureml.core import ScriptRunConfig
script_config=ScriptRunConfig(source_directory='.',script='hyperdrive.py',arguments=['--input-data',dataset.as_named_input('raw_data')],environment=envt,compute_target=cluster)

In [50]:
from azureml.train.hyperdrive import HyperDriveConfig,PrimaryMetricGoal
hyperconfig=HyperDriveConfig(run_config=script_config,hyperparameter_sampling=hyperparams,primary_metric_name='accuracy',
primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,max_total_runs=20,max_concurrent_runs=5)

In [51]:
experiment=Experiment(ws,'hyperdrive-run-KNN')
run=experiment.submit(config=hyperconfig)
run.wait_for_completion(show_output=True)

RunId: HD_4d032702-773d-4806-adaa-e0251eb9cdb5
Web View: https://ml.azure.com/runs/HD_4d032702-773d-4806-adaa-e0251eb9cdb5?wsid=/subscriptions/1db33695-8135-4616-9bb4-9574b401d454/resourcegroups/defaultsrg02/workspaces/hyperdive12&tid=6ad91895-de06-485e-bc51-fce126cc8530

Streaming azureml-logs/hyperdrive.txt

[2022-08-29T21:09:28.858356][GENERATOR][INFO]Trying to sample '5' jobs from the hyperparameter space
[2022-08-29T21:09:29.9802806Z][SCHEDULER][INFO]Scheduling job, id='HD_4d032702-773d-4806-adaa-e0251eb9cdb5_0' 
[2022-08-29T21:09:30.1462011Z][SCHEDULER][INFO]Scheduling job, id='HD_4d032702-773d-4806-adaa-e0251eb9cdb5_1' 
[2022-08-29T21:09:30.1667547Z][SCHEDULER][INFO]Scheduling job, id='HD_4d032702-773d-4806-adaa-e0251eb9cdb5_2' 
[2022-08-29T21:09:30.250253][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.
[2022-08-29T21:09:30.3103723Z][SCHEDULER][INFO]Scheduling job, id='HD_4d032702-773d-4806-adaa-e0251eb9cdb5_3' 
[2022-08-29T21

{'runId': 'HD_4d032702-773d-4806-adaa-e0251eb9cdb5',
 'target': 'cluster1',
 'status': 'Completed',
 'startTimeUtc': '2022-08-29T21:09:28.10294Z',
 'endTimeUtc': '2022-08-29T21:11:58.552302Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name":"accuracy","goal":"maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': '0a91b66c-48aa-47e7-8474-a69acde86d89',
  'user_agent': 'python/3.7.0 (Windows-10-10.0.22621-SP0) msrest/0.6.21 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.43.0',
  'space_size': '4'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'configuration': None,
  'attribution': None,
  'telemetryValues': {'amlClientType': 'azureml-sdk-train',
   'amlClientModule': '[Scrubbed]',
   'amlClientFunction': '[Scrubbed]',
   'tenantId': '6ad91895-de06-485e-bc51-fce126cc8530',
   'amlClientRequestId': 'ce74e3aa-b9fb-447a-8bc7-57de0b3d193b',
   'amlClientSessio

In [42]:
#retrieve best run
best_run=run.get_best_run_by_primary_metric()
#print("best Run ID",best_run.id)
print(best_run.get_metrics())

AttributeError: 'NoneType' object has no attribute 'get_metrics'

In [40]:
run.get_best_run_by_primary_metric()